In [ ]:
! pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! cp /content/drive/MyDrive/kaggle.json ~/
! mkdir ~/.kaggle
! mv ~/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [ ]:
! kaggle datasets download -d eldarsultanow/ml-bsd-dataset-integers

 97% 265M/274M [00:02<00:00, 100MB/s]
100% 274M/274M [00:03<00:00, 95.1MB/s]


In [ ]:
! unzip ml-bsd-dataset-integers.zip

Archive:  ml-bsd-dataset-integers.zip
  inflating: rank-vs-ap-integers.csv  


In [ ]:
import pandas as pd
import numpy as np
import sympy
from scipy.interpolate import interp1d
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.ensemble import GradientBoostingRegressor
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from sympy import prime, isprime, primerange
import random
import math
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score


In [ ]:
df = pd.read_csv("rank-vs-ap-integers.csv", index_col=False, header=None)


In [ ]:
new_columns = ['rank', 'con', 'root'] + [f'ap_{i}' for i in range(1, 1230)]

df.columns = new_columns

In [ ]:
df_2 = df.loc[df['rank'] == 2].sample(n=13616, random_state=1)
df_3 = df.loc[df['rank'] == 3]
df_3_2 = pd.concat([df_3, df_2])
df_1 = df.loc[df['rank'] == 1].sample(n=42290, random_state=1)
df_0 = df.loc[df['rank'] == 0]
df_0_1 = pd.concat([df_1, df_0])

df_sub = pd.concat([df_0_1, df_3_2])
df_sub['rank'].value_counts()

1    42290
0    42290
3    13616
2    13616
Name: rank, dtype: int64

In [ ]:
df_sub = df_sub[~df_sub['rank'].isin([4,5,6])]

In [ ]:
feature_col = [col for col in df.columns if col.startswith('ap_')]
X = df_sub[ feature_col]
y = df_sub['rank']

In [ ]:
print(X)

        ap_1  ap_2  ap_3  ap_4  ap_5  ap_6  ap_7  ap_8  ap_9  ap_10  ...  \
76492     -1     1     0     4    -4     5    -2     2    -8     -6  ...   
62660     -1     0     0     2     6     2    -1     4     3     -6  ...   
26131      0     0     4     1     0     3     2     0     7      5  ...   
74087     -1    -2     2     2     4     3     0     5     8      0  ...   
8084       2     1     2    -1     4    -6     5     4    -8      3  ...   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...    ...  ...   
117731    -1     2     1     4    -2     5     0    -3     2      9  ...   
133602    -1     1    -1    -4     4     4     4    -3    -3      4  ...   
126113    -1    -1    -1    -4    -4    -2    -2     2     8      3  ...   
113982    -1     2     0     1    -4     5    -6     4     1     -5  ...   
120408    -2     0     2     0     3     2     5    -8    -1      6  ...   

        ap_1220  ap_1221  ap_1222  ap_1223  ap_1224  ap_1225  ap_1226  \
76492        5

In [ ]:
top_features = ['ap_4', 'ap_3', 'ap_5', 'ap_6', 'ap_7', 'ap_8', 'ap_9', 'ap_10', 'ap_11', 'ap_12', 'ap_13', 'ap_14', 'ap_16', 'ap_15', 'ap_17', 'ap_18', 'ap_20', 'ap_19', 'ap_21', 'ap_23', 'ap_22', 'ap_24', 'ap_26', 'ap_29', 'ap_27', 'ap_28', 'ap_30', 'ap_25', 'ap_31', 'ap_32', 'ap_33', 'ap_36', 'ap_37', 'ap_43', 'ap_34', 'ap_35', 'ap_38', 'ap_40', 'ap_39', 'ap_44', 'ap_47', 'ap_42', 'ap_49', 'ap_46', 'ap_45', 'ap_48', 'ap_52', 'ap_53', 'ap_61', 'ap_62']


In [ ]:
from sympy import primepi

def get_r_sequence(N:int):
  pi_N = primepi(N)
  res = [None]*pi_N

  for i in range(1,pi_N+1):
    res[i-1] = -1 + i*2/pi_N
  return res

print(len(get_r_sequence(2000)))
print(get_r_sequence(2000))

303
[-301/303, -299/303, -99/101, -295/303, -293/303, -97/101, -289/303, -287/303, -95/101, -283/303, -281/303, -93/101, -277/303, -275/303, -91/101, -271/303, -269/303, -89/101, -265/303, -263/303, -87/101, -259/303, -257/303, -85/101, -253/303, -251/303, -83/101, -247/303, -245/303, -81/101, -241/303, -239/303, -79/101, -235/303, -233/303, -77/101, -229/303, -227/303, -75/101, -223/303, -221/303, -73/101, -217/303, -215/303, -71/101, -211/303, -209/303, -69/101, -205/303, -203/303, -67/101, -199/303, -197/303, -65/101, -193/303, -191/303, -63/101, -187/303, -185/303, -61/101, -181/303, -179/303, -59/101, -175/303, -173/303, -57/101, -169/303, -167/303, -55/101, -163/303, -161/303, -53/101, -157/303, -155/303, -51/101, -151/303, -149/303, -49/101, -145/303, -143/303, -47/101, -139/303, -137/303, -45/101, -133/303, -131/303, -43/101, -127/303, -125/303, -41/101, -121/303, -119/303, -39/101, -115/303, -113/303, -37/101, -109/303, -107/303, -35/101, -103/303, -1/3, -33/101, -97/303, -95/

In [ ]:
primes = [-301/303, -299/303, -99/101, -295/303, -293/303, -97/101, -289/303, -287/303, -95/101, -283/303, -281/303, -93/101, -277/303, -275/303, -91/101, -271/303, -269/303, -89/101, -265/303, -263/303, -87/101, -259/303, -257/303, -85/101, -253/303, -251/303, -83/101, -247/303, -245/303, -81/101, -241/303, -239/303, -79/101, -235/303, -233/303, -77/101, -229/303, -227/303, -75/101, -223/303, -221/303, -73/101, -217/303, -215/303, -71/101, -211/303, -209/303, -69/101, -205/303, -203/303, -67/101, -199/303, -197/303, -65/101, -193/303, -191/303, -63/101, -187/303, -185/303, -61/101, -181/303, -179/303, -59/101, -175/303, -173/303, -57/101, -169/303, -167/303, -55/101, -163/303, -161/303, -53/101, -157/303, -155/303, -51/101, -151/303, -149/303, -49/101, -145/303, -143/303, -47/101, -139/303, -137/303, -45/101, -133/303, -131/303, -43/101, -127/303, -125/303, -41/101, -121/303, -119/303, -39/101, -115/303, -113/303, -37/101, -109/303, -107/303, -35/101, -103/303, -1/3, -33/101, -97/303, -95/303, -31/101, -91/303, -89/303, -29/101, -85/303, -83/303, -27/101, -79/303, -77/303, -25/101, -73/303, -71/303, -23/101, -67/303, -65/303, -21/101, -61/303, -59/303, -19/101, -55/303, -53/303, -17/101, -49/303, -47/303, -15/101, -43/303, -41/303, -13/101, -37/303, -35/303, -11/101, -31/303, -29/303, -9/101, -25/303, -23/303, -7/101, -19/303, -17/303, -5/101, -13/303, -11/303, -3/101, -7/303, -5/303, -1/101, -1/303, 1/303, 1/101, 5/303, 7/303, 3/101, 11/303, 13/303, 5/101, 17/303, 19/303, 7/101, 23/303, 25/303, 9/101, 29/303, 31/303, 11/101, 35/303, 37/303, 13/101, 41/303, 43/303, 15/101, 47/303, 49/303, 17/101, 53/303, 55/303, 19/101, 59/303, 61/303, 21/101, 65/303, 67/303, 23/101, 71/303, 73/303, 25/101, 77/303, 79/303, 27/101, 83/303, 85/303, 29/101, 89/303, 91/303, 31/101, 95/303, 97/303, 33/101, 1/3, 103/303, 35/101, 107/303, 109/303, 37/101, 113/303, 115/303, 39/101, 119/303, 121/303, 41/101, 125/303, 127/303, 43/101, 131/303, 133/303, 45/101, 137/303, 139/303, 47/101, 143/303, 145/303, 49/101, 149/303, 151/303, 51/101, 155/303, 157/303, 53/101, 161/303, 163/303, 55/101, 167/303, 169/303, 57/101, 173/303, 175/303, 59/101, 179/303, 181/303, 61/101, 185/303, 187/303, 63/101, 191/303, 193/303, 65/101, 197/303, 199/303, 67/101, 203/303, 205/303, 69/101, 209/303, 211/303, 71/101, 215/303, 217/303, 73/101, 221/303, 223/303, 75/101, 227/303, 229/303, 77/101, 233/303, 235/303, 79/101, 239/303, 241/303, 81/101, 245/303, 247/303, 83/101, 251/303, 253/303, 85/101, 257/303, 259/303, 87/101, 263/303, 265/303, 89/101, 269/303, 271/303, 91/101, 275/303, 277/303, 93/101, 281/303, 283/303, 95/101, 287/303, 289/303, 97/101, 293/303, 295/303, 99/101, 299/303, 301/303, 1]


In [ ]:
abs_prime = [abs(prime) for prime in primes]

In [ ]:
ap_cols = [col for col in df_sub.columns if col.startswith("ap_")]
df_sub[ap_cols] = df_sub[ap_cols].replace(0, np.nan)
df_sub[ap_cols] = df_sub[ap_cols].fillna(df_sub[ap_cols].mean())

In [ ]:
for i in range(len(abs_prime)):
    ap_col = f"ap_{i+1}"
    df_sub[ap_col].dropna(inplace=True)
    if (df_sub[ap_col] == 0).any():
        df_sub[ap_col].replace(0, df_sub[ap_col].mean(), inplace=True)
    df_sub[ap_col] = df_sub[ap_col].apply(lambda x, p=abs_prime[i]: x / math.sqrt(p))


In [ ]:
print(X)

        root      ap_4      ap_3      ap_5      ap_6      ap_7      ap_8  \
76492     -1  4.053874 -0.063887 -4.067687  5.102051 -2.047870  2.054993   
62660     -1  2.026937 -0.063887  6.101530  2.040821 -1.023935  4.109986   
26131     -1  1.013469  4.040202 -0.071751  3.061231  2.047870 -0.089960   
74087     -1  2.026937  2.020101  4.067687  3.061231 -0.078319  5.137483   
8084      -1 -1.013469  2.020101  4.067687 -6.122462  5.119675  4.109986   
...      ...       ...       ...       ...       ...       ...       ...   
117731     1  4.053874  1.010051 -2.033843  5.102051 -0.078319 -3.082490   
133602     1 -4.053874 -1.010051  4.067687  4.081641  4.095740 -3.082490   
126113     1 -4.053874 -1.010051 -4.067687 -2.040821 -2.047870  2.054993   
113982     1  1.013469 -0.063887 -4.067687  5.102051 -6.143610  4.109986   
120408     1 -0.069235  2.020101  3.050765  2.040821  5.119675 -8.219972   

            ap_9     ap_10     ap_11  ...      ap_47      ap_42      ap_49  \
76492  -8

In [ ]:
feature_col = [col for col in df.columns if col.startswith('ap_')]
X = df_sub[['root']+ top_features]
y = df_sub['rank']

In [ ]:
print(y)

In [ ]:
from sklearn.decomposition import FastICA
import xgboost as xgb
n_components = 4  
ica = FastICA(n_components=n_components, random_state=10)
independent_components = ica.fit_transform(X)

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


In [ ]:
print(independent_components)

[[-5.27768629e-03 -3.18767549e-03 -2.88786430e-03 -1.03682880e-03]
 [ 2.57015288e-03 -1.02905882e-03 -1.38783075e-04 -1.42194472e-03]
 [-2.65067584e-05  9.85003435e-04 -4.71550934e-03 -2.34374540e-03]
 ...
 [ 7.18808537e-04  3.14323841e-03 -9.61742680e-04  1.52844499e-03]
 [ 1.63439402e-05 -1.56036641e-03 -1.50256221e-03 -6.65234003e-04]
 [ 8.27282140e-04 -5.16172701e-03 -1.63551401e-03 -5.12989004e-03]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=10)
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_train, y_train, test_size=0.5, random_state=10)

In [ ]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

val_predictions = xgb_model.predict(X_test_val)
val_accuracy = np.sum(val_predictions == y_test_val) / len(y_test_val)
print("Validation Accuracy:", val_accuracy)

# Evaluate the XGBoost model on test set
test_predictions = xgb_model.predict(X_test)
test_accuracy = np.sum(test_predictions == y_test) / len(y_test)
print("Test Accuracy:", test_accuracy)

Validation Accuracy: 0.9623468383865487
Test Accuracy: 0.8012610302885762


In [ ]:
import lightgbm as lgb

clf = lgb.LGBMClassifier(n_estimators=100,max_depth=50,learning_rate= 0.1)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test_val)

print("RMSE: {:.4f}".format(np.sqrt(mean_squared_error(y_test_val, y_pred))))
print("R^2 Score: {:.4f}".format(r2_score(y_test_val, y_pred)))
print("ACC Score: {:.4f}".format(accuracy_score(y_test_val, y_pred)))

RMSE: 0.6053
R^2 Score: 0.6243
ACC Score: 0.9084


In [ ]:
import time
from sklearn.metrics import confusion_matrix, classification_report
def forest_test(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=10)

    start = time.process_time()
    trainedforest = RandomForestClassifier(n_estimators=100).fit(X_train,y_train)
    print(time.process_time() - start)
    predictionforest = trainedforest.predict(X_test)
    print(confusion_matrix(y_test,predictionforest))
    print(classification_report(y_test,predictionforest))

In [ ]:
#from sklearn.manifold import LocallyLinearEmbedding

#embedding = LocallyLinearEmbedding(n_components=4)
#X_lle = embedding.fit_transform(X)

forest_test(X, y)

21.881620767999948
[[25286     0    52     0]
 [    0 25187     0   208]
 [ 7938     0   144     0]
 [    0  6871     0  1402]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.86     25338
           1       0.79      0.99      0.88     25395
           2       0.73      0.02      0.03      8082
           3       0.87      0.17      0.28      8273

    accuracy                           0.78     67088
   macro avg       0.79      0.54      0.51     67088
weighted avg       0.78      0.78      0.70     67088



In [ ]:

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=1)

# run an LDA and use it to transform the features
X_lda = lda.fit(X, y).transform(X)
print('Original number of features:', X.shape[1])
print('Reduced number of features:', X_lda.shape[1])

Original number of features: 51
Reduced number of features: 1


In [ ]:
forest_test(X_lda, y)

17.592462361999992
[[11296 10206  2633  1203]
 [10422 10104  2901  1968]
 [ 2457  2859  1272  1494]
 [ 1258  2023  1571  3421]]
              precision    recall  f1-score   support

           0       0.44      0.45      0.44     25338
           1       0.40      0.40      0.40     25395
           2       0.15      0.16      0.15      8082
           3       0.42      0.41      0.42      8273

    accuracy                           0.39     67088
   macro avg       0.36      0.35      0.35     67088
weighted avg       0.39      0.39      0.39     67088



In [ ]:
#from sklearn.manifold import TSNE

#start = time.process_time()
#tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
#X_tsne = tsne.fit_transform(X)
#print(time.process_time() - start)